# Analysis writeup revisited

Author: Luis Eduardo San Martin

This writeup is a revision of the former analysis writeup I submitted for assignment 3.

## The problem

Once you've set up the improved pipeline, you should apply it to solve the following problem:

The problem is to predict if a project on donorschoose will not get fully funded within 60 days of posting. This prediction is being done at the time of posting so you can only use data available to you at that time. The data is a file that has one row for each project posted with a column for "date_posted" (the date the project was posted) and a column for "datefullyfunded" (the date the project was fully funded - assumption for this assignment is that all projects were fully funded eventually). The task is to predict if a project on donorschoose will not get fully funded within 60 days of posting.

The data spans Jan 1, 2012 to Dec 31, 2013. You should have your validation/test set be a rolling window of 6 months (which should give you three test sets). The training sets should be everything from 1/1/12 to the beginning of the test set.

The code should produce a table with results across train test splits over time and performance metrics (baseline, precision and recall at different thresholds 1%, 2%, 5%, 10%, 20%, 30%, 50% and AUC_ROC)

## My (new) solution

For this solution, I'll rely on the functions I wrote for my Machine Learning pipeline.

### Reading the data

In [1]:
%load_ext autoreload
%autoreload 2
import pipeline
import pandas as pd

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pipeline.read('data/projects_2012_2013.csv')
df

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_metro,school_district,...,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level,grade_level,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,date_posted,datefullyfunded
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,urban,Pershing Elem Network,...,Visual Arts,Music & The Arts,Supplies,highest poverty,Grades PreK-2,1498.61,31.0,f,4/14/13,5/2/13
1,0000fa3aa8f6649abab23615b546016d,2a578595fe351e7fce057e048c409b18,3432ed3d4466fac2f2ead83ab354e333,6.409801e+10,34.296596,-119.296596,Ventura,CA,urban,Ventura Unif School District,...,Literature & Writing,Literacy & Language,Books,highest poverty,Grades 3-5,282.47,28.0,t,4/7/12,4/18/12
2,000134f07d4b30140d63262c871748ff,26bd60377bdbffb53a644a16c5308e82,dc8dcb501c3b2bb0b10e9c6ee2cd8afd,6.227100e+10,34.078625,-118.257834,Los Angeles,CA,urban,Los Angeles Unif Sch Dist,...,Social Sciences,History & Civics,Technology,high poverty,Grades 3-5,1012.38,56.0,f,1/30/12,4/15/12
3,0001f2d0b3827bba67cdbeaa248b832d,15d900805d9d716c051c671827109f45,8bea7e8c6e4279fca6276128db89292e,3.600090e+11,40.687286,-73.988217,Brooklyn,NY,urban,New York City Dept Of Ed,...,NaN,NaN,Books,high poverty,Grades PreK-2,175.33,23.0,f,10/11/12,12/5/12
4,0004536db996ba697ca72c9e058bfe69,400f8b82bb0143f6a40b217a517fe311,fbdefab6fe41e12c55886c610c110753,3.606870e+11,40.793018,-73.205635,Central Islip,NY,suburban,Central Islip Union Free SD,...,Literature & Writing,Literacy & Language,Technology,high poverty,Grades PreK-2,3591.11,150.0,f,1/8/13,3/25/13
5,00049ec8ca1f2d08cb13cab31b0b85ec,7149611553c700de9a6099f8a9ce598b,462a5fd93cf9fb5d41eecfd2ea860b19,2.621150e+11,42.740157,-84.525821,Lansing,MI,urban,Lansing School District,...,NaN,NaN,Other,highest poverty,Grades 3-5,475.85,15.0,f,11/30/12,2/26/13
6,0004d2fdbb571237fa53a97e7691440b,926671e209fb977bd5123145c1848ad1,1a994778027ab086dc58ec3b47f74ff0,4.047200e+10,33.059361,-112.037727,Maricopa,AZ,rural,Maricopa Unif Sch District 20,...,NaN,NaN,Supplies,high poverty,Grades 3-5,390.65,37.0,f,3/26/13,4/17/13
7,0004ee26667e751dd51384eb9f30c72e,abe4dabb7864f4c548d230cf9070e03f,8409f70bcd81bc06e4b9efca68eed8f6,6.280501e+10,37.761958,-122.193209,Oakland,CA,urban,Oakland Unified School Dist,...,NaN,NaN,Books,highest poverty,Grades 9-12,3877.20,30.0,f,2/28/13,3/10/13
8,0006a31d45f8d52d217e7c5b55c11f37,3b5fada1ad0e339acc669829071320c4,c6a033f9349ea70659c1891b119680ed,2.307320e+11,44.096641,-70.191734,Lewiston,ME,urban,Lewiston Public Schools,...,NaN,NaN,Technology,high poverty,Grades 3-5,838.75,25.0,f,8/21/13,9/13/13
9,0008ac907bf237a15a959244205d3ee5,92527a5ac5fe946ed1961fb2e1de8cc5,23e34f5d2e2940684269cffe35741598,6.271800e+10,34.381832,-118.531837,Newhall,CA,suburban,Newhall School District,...,Literacy,Literacy & Language,Technology,highest poverty,Grades PreK-2,1477.44,24.0,f,10/3/12,11/3/12


### Exploring the dataframe

* Column types

In [3]:
pipeline.columns_types(df)

projectid                                  object
teacher_acctid                             object
schoolid                                   object
school_ncesid                             float64
school_latitude                           float64
school_longitude                          float64
school_city                                object
school_state                               object
school_metro                               object
school_district                            object
school_county                              object
school_charter                             object
school_magnet                              object
teacher_prefix                             object
primary_focus_subject                      object
primary_focus_area                         object
secondary_focus_subject                    object
secondary_focus_area                       object
resource_type                              object
poverty_level                              object


Most of our data is discrete, with a few exceptions.

* Number of observations

In [4]:
pipeline.count_obs(df)

124976


This number is long enough for generating supervised learning models.

* Counting duplicates for all variables

In [5]:
pipeline.duplicates(df)

0


Good, no duplicated observations.

* Counting duplicates -- project ID

In [6]:
pipeline.duplicates_in_columns(df, ['projectid'])

0


No duplicates in project ID.

* Missing values

In [7]:
pipeline.count_missings(df)

projectid has 0.0 % of missing data points
teacher_acctid has 0.0 % of missing data points
schoolid has 0.0 % of missing data points
school_ncesid has 7.38781846114454 % of missing data points
school_latitude has 0.0 % of missing data points
school_longitude has 0.0 % of missing data points
school_city has 0.0 % of missing data points
school_state has 0.0 % of missing data points
school_metro has 12.181538855460248 % of missing data points
school_district has 0.1376264242734605 % of missing data points
school_county has 0.0 % of missing data points
school_charter has 0.0 % of missing data points
school_magnet has 0.0 % of missing data points
teacher_prefix has 0.0 % of missing data points
primary_focus_subject has 0.012002304442452951 % of missing data points
primary_focus_area has 0.012002304442452951 % of missing data points
secondary_focus_subject has 32.45103059787479 % of missing data points
secondary_focus_area has 32.45103059787479 % of missing data points
resource_type has 0.01

We see some missing values in some of the variables we could use as features. Here is how we'll treat them:

1. We'll leave out of the analysis the variable `school_ncesid`. It is a school ID and we won't need it to train/test our algorithm.
2. We'll also leave out `school_metro` and `school_district`. Being discrete school administration/location variables, their information is probable very correlated with other variables, such as `school_city` and `school_county`.
3. We'll assume that the missings in `primary_focus_subject`, `primary_focus_area`, `secondary_focus_subject`,  `secondary_focus_area`, `resource_type` and `grade_level` are actually conveying some information. Thus, we'll impute the label 'None' in these missing values.
4. Finally, in `students_reached` we'll impute the median. Notice that this variable in continuous.

Also, it's important to note that the variables we'll use to generate our label and the train/test splits don't have missing values (`date_posted`, `datefullyfunded`).

* Checking some basic statistics:

In [8]:
pipeline.describe(df)

count                               124976
unique                              124976
top       6e68a5151f21995ef8042ce4478a71c1
freq                                     1
Name: projectid, dtype: object
count                               124976
unique                               77013
top       214acf23d183dfa2f1dc16e7b3658320
freq                                    63
Name: teacher_acctid, dtype: object
count                               124976
unique                               29947
top       10179fd362d7b8cf0e89baa1ca3025bb
freq                                   193
Name: schoolid, dtype: object
count    1.157430e+05
mean     2.448448e+11
std      1.644728e+11
min      1.000050e+10
25%      6.344101e+10
50%      2.200870e+11
75%      3.704880e+11
max      6.100010e+11
Name: school_ncesid, dtype: float64
count    124976.000000
mean         36.827284
std           4.963669
min          18.249140
25%          33.872504
50%          36.617410
75%          40.676156
max          6

We can see the following, for our subsequent data preparation:

Identifier variables:

* `project_id`
* `teacher_acctid`
* `schoolid`
* `school_ncesid`

Discrete variables:
 
* `school_magnet`
* `school_charter`
* `eligible_double_your_impact_match`
 
Categorical variables:
 
* `school_city`
* `school_state`
* `school_metro`
* `school_district`
* `school_county`
* `school_charter`
* `teacher_prefix`
* `primary_focus_subject`
* `primary_focus_area`
* `secondary_focus_subject`
* `secondary_focus_area`
* `resource_type`
* `poverty_level`
* `grade_level`

Continuous variables:
 
* `school_latitude`
* `school_longitude`
* `total_price_including_optional_support`
* `students_reached`

Date variables:

* `date_posted`
* `datefullyfunded`

### Preparing the dataframes

Here's what we'll do:

1. Transform date features into date types
2. Add dummies for each value of every categorical variable
3. Generate the label attribute
4. Generate the train/test sets
5. Discretize categorical variables and impute missing values in each train/test set
6. Define the columns we'll use as features


So:

1. Transforming date features into date types:

In [9]:
date_vars = ['date_posted', 'datefullyfunded']
for col in date_vars:
    pipeline.to_date(df, col)

2. Adding dummies for each value of every categorical variable

* Replacing missing values with the label 'None':

In [10]:
none_label_cols = ['primary_focus_subject', 'primary_focus_area', 'secondary_focus_subject', 'secondary_focus_area', \
                   'resource_type', 'grade_level']
for col in none_label_cols:
    pipeline.fill_nas_other(df, col, 'None')

* Now we generate the dummies

In [11]:
categorical = ['school_state', 'school_metro', 'teacher_prefix', 'primary_focus_area', \
               'secondary_focus_area', 'resource_type', 'poverty_level', 'grade_level']
for col in categorical:
    pipeline.create_dummies(df, col)

3. Generating the label attribute -- notice that we want to predict if a project will not get funding within the first 60 days of having being posted.

In [12]:
days = 60
pipeline.create_time_label(df, 'date_posted', 'datefullyfunded', days)
df[date_vars + ['label']]

,date_posted,datefullyfunded,label
0,2013-04-14,2013-05-02,0
1,2012-04-07,2012-04-18,0
2,2012-01-30,2012-04-15,1
3,2012-10-11,2012-12-05,0
4,2013-01-08,2013-03-25,1
5,2012-11-30,2013-02-26,1
6,2013-03-26,2013-04-17,0
7,2013-02-28,2013-03-10,0
8,2013-08-21,2013-09-13,0
9,2012-10-03,2012-11-03,0


4. Generating the train/test sets. We also store them in a dictionary we'll use for generating the evaluation table.

In [13]:
months = 6
time_feature = 'date_posted'
threshold1 = '06/30/2012'
threshold2 = '12/31/2012'
threshold3 = '06/30/2013'

print('\nHoldout 1')
df_train1, df_test1 = pipeline.time_based_split(df, time_feature, threshold1, days, months)
print('\nHoldout 2')
df_train2, df_test2 = pipeline.time_based_split(df, time_feature, threshold2, days, months)
print('\nHoldout 3')
df_train3, df_test3 = pipeline.time_based_split(df, time_feature, threshold3, days, months)

sets = [df_train1, df_train2, df_train3, df_test1, df_test2, df_test3]


Holdout 1
train upper threshold: 2012-05-01 00:00:00
Notice that we leave a gap of 60 days
test lower threshold: 2012-06-30 00:00:00
test upper threshold: 2012-12-30 00:00:00

Holdout 2
train upper threshold: 2012-11-01 00:00:00
Notice that we leave a gap of 60 days
test lower threshold: 2012-12-31 00:00:00
test upper threshold: 2013-06-30 00:00:00

Holdout 3
train upper threshold: 2013-05-01 00:00:00
Notice that we leave a gap of 60 days
test lower threshold: 2013-06-30 00:00:00
test upper threshold: 2013-12-30 00:00:00


5. Discretizing categorical variables and imputing missing values in each train/test set

* Replacing missing values with the median:

In [14]:
median_cols = ['students_reached']
for dataset in sets:
    for col in median_cols:
        pipeline.fill_nas_median(dataset, col)

* Transforming discrete variables into 0/1:

In [15]:
discrete_vars = ['school_magnet', 'school_charter', 'eligible_double_your_impact_match']
for dataset in sets:
    for col in discrete_vars:
        pipeline.discrete_0_1(dataset, col, 'f', 't')

* Creating dummies for each quartile of the continuous variables we'll use as features: (notice that we won't use `students_reached` as a feature because according to the [data dictionary](https://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/data) that variable is available after a project is funded)

In [16]:
continuous = ['total_price_including_optional_support']
for dataset in sets:
    for col in continuous:
        pipeline.discretize(dataset, col)
        pipeline.create_dummies(dataset, col + '_quartile')

5. Defining the columns we'll use as features and our datasets dictionary:

In [17]:
features = ['school_charter', 'school_magnet', 'eligible_double_your_impact_match', 'school_state_IL', \
            'school_state_CA', 'school_state_NY', 'school_state_MI', 'school_state_AZ', 'school_state_ME', \
            'school_state_MO', 'school_state_FL', 'school_state_KY', 'school_state_GA', 'school_state_TX', \
            'school_state_IN', 'school_state_NC', 'school_state_SC', 'school_state_CT', 'school_state_OH', \
            'school_state_MN', 'school_state_WV', 'school_state_WA', 'school_state_TN', 'school_state_OK', \
            'school_state_DC', 'school_state_MD', 'school_state_MS', 'school_state_ID', 'school_state_MA', \
            'school_state_IA', 'school_state_AK', 'school_state_WI', 'school_state_NV', 'school_state_LA', \
            'school_state_NE', 'school_state_CO', 'school_state_KS', 'school_state_OR', 'school_state_VA', \
            'school_state_PA', 'school_state_NJ', 'school_state_SD', 'school_state_MT', 'school_state_NH', \
            'school_state_AR', 'school_state_UT', 'school_state_WY', 'school_state_ND', 'school_state_HI', \
            'school_state_AL', 'school_state_RI', 'school_state_DE', 'school_state_NM', 'school_state_VT', \
            'school_metro_urban', 'school_metro_suburban', 'school_metro_rural', 'school_metro_nan', \
            'teacher_prefix_Mrs.', 'teacher_prefix_Ms.', 'teacher_prefix_Mr.', 'teacher_prefix_Dr.', \
            'primary_focus_area_Math & Science', 'primary_focus_area_History & Civics', \
            'primary_focus_area_Literacy & Language', 'primary_focus_area_Applied Learning', \
            'primary_focus_area_Music & The Arts', 'primary_focus_area_Health & Sports', \
            'primary_focus_area_Special Needs', 'primary_focus_area_None', 'secondary_focus_area_Music & The Arts', \
            'secondary_focus_area_Literacy & Language', 'secondary_focus_area_History & Civics', \
            'secondary_focus_area_None', 'secondary_focus_area_Health & Sports', \
            'secondary_focus_area_Math & Science', 'secondary_focus_area_Special Needs', \
            'secondary_focus_area_Applied Learning', 'resource_type_Supplies', 'resource_type_Books', \
            'resource_type_Technology', 'resource_type_Other', 'resource_type_Trips', 'resource_type_Visitors', \
            'resource_type_None', 'grade_level_None', 'poverty_level_highest poverty', 'poverty_level_high poverty', \
            'poverty_level_low poverty', 'poverty_level_moderate poverty', 'grade_level_Grades PreK-2', \
            'grade_level_Grades 3-5', 'grade_level_Grades 9-12', 'grade_level_Grades 6-8', \
            'total_price_including_optional_support_quartile_4.0', \
            'total_price_including_optional_support_quartile_1.0', \
            'total_price_including_optional_support_quartile_2.0', \
            'total_price_including_optional_support_quartile_3.0']
label = 'label'
datasets = {'Holdout 1: ' + threshold1: [df_train1, df_test1],
            'Holdout 2: ' + threshold2: [df_train2, df_test2],
            'Holdout 3: ' + threshold3: [df_train3, df_test3]}

### Generating the evaluation table

We'll use two global variables in our pipeline script where we have defined a dictionary of classifiers and a dictionary of parameters we could use (`pipeline.CLASSIFIERS` and `pipeline.PARAMETERS`), but notice that the `evaluation_table` function can also be used with any dictionary of classifiers and parameters.

In [18]:
fractions = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]

In [19]:
table = pipeline.evaluation_table(pipeline.CLASSIFIERS, pipeline.PARAMETERS_MID, datasets, fractions, features, label)

Dataset: Holdout 2: 12/31/2012
Predicting every data point's value to be 0, the accuracy is 68.5 %

Running model 1 out of 324
Progress: 0.3 %

Running model 2 out of 324
Progress: 0.6 %

Running model 3 out of 324
Progress: 0.9 %

Running model 4 out of 324
Progress: 1.2 %

Running model 5 out of 324
Progress: 1.5 %

Running model 6 out of 324
Progress: 1.9 %

Running model 7 out of 324
Progress: 2.2 %

Running model 8 out of 324
Progress: 2.5 %

Running model 9 out of 324
Progress: 2.8 %

Running model 10 out of 324
Progress: 3.1 %

Running model 11 out of 324
Progress: 3.4 %

Running model 12 out of 324
Progress: 3.7 %

Running model 13 out of 324
Progress: 4.0 %

Running model 14 out of 324
Progress: 4.3 %

Running model 15 out of 324
Progress: 4.6 %

Running model 16 out of 324
Progress: 4.9 %

Running model 17 out of 324
Progress: 5.2 %

Running model 18 out of 324
Progress: 5.6 %

Running model 19 out of 324
Progress: 5.9 %

Running model 20 out of 324
Progress: 6.2 %

Running m


Running model 175 out of 324
Progress: 54.0 %

Running model 176 out of 324
Progress: 54.3 %

Running model 177 out of 324
Progress: 54.6 %

Running model 178 out of 324
Progress: 54.9 %

Running model 179 out of 324
Progress: 55.2 %

Running model 180 out of 324
Progress: 55.6 %

Running model 181 out of 324
Progress: 55.9 %

Running model 182 out of 324
Progress: 56.2 %

Running model 183 out of 324
Progress: 56.5 %

Running model 184 out of 324
Progress: 56.8 %

Running model 185 out of 324
Progress: 57.1 %

Running model 186 out of 324
Progress: 57.4 %

Running model 187 out of 324
Progress: 57.7 %

Running model 188 out of 324
Progress: 58.0 %

Running model 189 out of 324
Progress: 58.3 %

Running model 190 out of 324
Progress: 58.6 %

Running model 191 out of 324
Progress: 59.0 %

Running model 192 out of 324
Progress: 59.3 %

Running model 193 out of 324
Progress: 59.6 %

Running model 194 out of 324
Progress: 59.9 %

Running model 195 out of 324
Progress: 60.2 %

Running mode

In [20]:
table

,Exact classifier,classifier,parameters,dataset,baseline,precision_at_0.01,precision_at_0.02,precision_at_0.05,precision_at_0.1,precision_at_0.2,precision_at_0.3,precision_at_0.5,recall_at_0.01,recall_at_0.02,recall_at_0.05,recall_at_0.1,recall_at_0.2,recall_at_0.3,recall_at_0.5,AUC ROC
0,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 10, 'n_jobs': 10, 'max_sample...",Holdout 2: 12/31/2012,0.685059,0.516279,0.554524,0.527340,0.518536,0.481584,0.450502,0.415678,0.016328,0.035157,0.083701,0.164607,0.305825,0.429097,0.659900,0.659041
1,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 100, 'n_jobs': 10, 'max_sampl...",Holdout 2: 12/31/2012,0.685059,0.576744,0.538283,0.533828,0.527340,0.489460,0.458069,0.423925,0.018241,0.034128,0.084731,0.167402,0.310827,0.436305,0.672992,0.669567
2,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 10, 'n_jobs': 10, 'max_sample...",Holdout 2: 12/31/2012,0.685059,0.483721,0.480278,0.481001,0.478684,0.472319,0.446178,0.413176,0.015299,0.030450,0.076346,0.151956,0.299941,0.424978,0.655928,0.654921
3,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 100, 'n_jobs': 10, 'max_sampl...",Holdout 2: 12/31/2012,0.685059,0.553488,0.501160,0.525487,0.525023,0.485522,0.449421,0.423832,0.017505,0.031774,0.083407,0.166667,0.308326,0.428067,0.672845,0.667621
4,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 10, 'n_jobs': 10, 'max_sample...",Holdout 2: 12/31/2012,0.685059,0.493023,0.459397,0.487488,0.515755,0.443595,0.433668,0.411694,0.015593,0.029126,0.077376,0.163725,0.281701,0.413063,0.653575,0.652081
5,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 100, 'n_jobs': 10, 'max_sampl...",Holdout 2: 12/31/2012,0.685059,0.511628,0.545244,0.516219,0.525487,0.495251,0.451892,0.424574,0.016181,0.034569,0.081936,0.166814,0.314504,0.430421,0.674022,0.669523
6,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 10, 'n_jobs': 10, 'max_sample...",Holdout 2: 12/31/2012,0.685059,0.455814,0.498840,0.502317,0.483318,0.495946,0.451274,0.413454,0.014416,0.031627,0.079729,0.153427,0.314946,0.429832,0.656370,0.660362
7,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 100, 'n_jobs': 10, 'max_sampl...",Holdout 2: 12/31/2012,0.685059,0.530233,0.535963,0.497683,0.519462,0.486449,0.450965,0.421053,0.016770,0.033981,0.078994,0.164901,0.308914,0.429538,0.668432,0.666117
8,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 10, 'n_jobs': 10, 'max_sample...",Holdout 2: 12/31/2012,0.685059,0.483721,0.512761,0.515292,0.494903,0.482511,0.453745,0.417346,0.015299,0.032510,0.081789,0.157105,0.306414,0.432186,0.662548,0.661887
9,BaggingClassifier(base_estimator=DecisionTreeC...,Bagging,"{'n_estimators': 100, 'n_jobs': 10, 'max_sampl...",Holdout 2: 12/31/2012,0.685059,0.586047,0.549884,0.557924,0.521779,0.496641,0.468417,0.426798,0.018535,0.034863,0.088555,0.165637,0.315387,0.446161,0.677552,0.677530


In [21]:
table.to_csv('evaluation_table.csv')